## «Человек в цикле: контроль над действиями агента»

**Цель:** научить агента приостанавливать выполнение перед критическими действиями (например, отправкой email) и ждать одобрения, отклонения или редактирования от человека.

Это жизненно важно для production, где агент не должен слать письма, тратить деньги или менять данные без разрешения.

### Часть 1: Настройка агента с HumanInTheLoopMiddleware


**Зачем это нужно?**

- Агенты могут вызывать потенциально опасные инструменты (send_email, pay_invoice, delete_user).
- Без контроля — риск ошибок, утечек, финансовых потерь.
- Человек должен иметь возможность одобрить, отклонить или отредактировать действие до его выполнения.

**Как работает HumanInTheLoopMiddleware?**

**Механизм:**
- Агент решает вызвать инструмент (например, send_email),
- Если инструмент в списке interrupt_on со значением True:
    - Выполнение приостанавливается,
    - В ответе появляется специальный ключ __interrupt__,
    - Система ждёт команды от человека через Command(resume=...).

In [ ]:
from langchain.tools import tool, ToolRuntime
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

load_dotenv()


OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="google/gemini-3-flash-preview",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

In [ ]:
@tool
def read_email(runtime: ToolRuntime) -> str:
    "Read an email from the given address."
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    return f"Email sent."

In [ ]:
class EmailState(AgentState):
    email: str

agent = create_agent(
    model=llm,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval"
        )
    ]
)

In [ ]:
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now in the same thread.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [ ]:
common_input = {
    "messages": [HumanMessage(
        content="Please read my email and send a response immediately. Send the reply now in the same thread."
    )],
    "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
}

In [ ]:
config_approve = {"configurable": {"thread_id": "approve_test"}}

response = agent.invoke(common_input, config=config_approve)

print("=== Прерывание (ожидается '__interrupt__') ===")
pprint(response.get('__interrupt__', 'ОШИБКА: прерывания нет!'))

from langgraph.types import Command
response = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=config_approve
)

print("\n=== Результат после APPROVE ===")
pprint(response)

In [ ]:
config_reject = {"configurable": {"thread_id": "reject_test"}}

response = agent.invoke(common_input, config=config_reject)

print("=== Прерывание (ожидается '__interrupt__') ===")
pprint(response.get('__interrupt__', 'ОШИБКА: прерывания нет!'))

response = agent.invoke(
    Command(resume={
        "decisions": [{
            "type": "reject",
            "message": "No please sign off - Your merciful leader, Seán."
        }]
    }),
    config=config_reject
)

print("\n=== Результат после REJECT ===")
pprint(response)

In [ ]:
config_edit = {"configurable": {"thread_id": "edit_test"}}

response = agent.invoke(common_input, config=config_edit)

print("=== Прерывание (ожидается '__interrupt__') ===")
pprint(response.get('__interrupt__', 'ОШИБКА: прерывания нет!'))

response = agent.invoke(
    Command(resume={
        "decisions": [{
            "type": "edit",
            "edited_action": {
                "name": "send_email",
                "args": {"body": "This is the last straw, you're fired!"}
            }
        }]
    }),
    config=config_edit
)

print("\n=== Результат после EDIT ===")
pprint(response)